## Analyse network data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import os

In [2]:
# Open attack.csv

df_attack = pd.read_csv('/home/johan/Desktop/PDSSC/Projet/data/dataset/Network datatset/csv/attack_3.csv', delimiter=',', header=0) 
# 5527409 rows attack_1
# 5159469 rows attack_2

In [3]:
df_attack.shape

(5527409, 16)

In [4]:
# Change the name of the columns by removing the spaces
df_attack.columns = df_attack.columns.str.replace(' ', '')

In [ ]:
# Open normal.csv 

df_normal = pd.read_csv('/home/johan/Desktop/PDSSC/Projet/data/dataset/Network datatset/csv/normal.csv', delimiter=',', header=0, nrows=5000000)

In [5]:
# Remove duplicates from both dataframes

df_attack1 = df_attack.drop_duplicates()
# df_normal = df_normal.drop_duplicates()

In [6]:
def get_columns_info(column_name):
    print('Column name: ', column_name)
    print('Number of unique values in attack: ', df_attack[column_name].nunique())
    print('Number of missing values in attack: ', df_attack1[column_name].isnull().sum())
    print('Number of values in attack: ', df_attack1[column_name].count())
    print('Values in attack: ', df_attack1[column_name].unique())

# Début Rapport d'analyse

problème avec ces colonne : 'ip_s', 'ip_d', 'sport', 'dport', 'flags', 'n_pkt_src', 'n_pkt_dst', 'modbus_fn', 'modbus_response'
raison : missing some values

problème avec modbus_fn, lot of missing value (153121 for attack1), useless column ? Même chose avec modbus_response : Colonne lié au proto modbus et donc pas toute les colonne

La colonne proto contient les protocoles ARP ICMP Modbus TCP pour attack mais seulement Modbus et TCP pour le cas normal

Problème de typage : 'sport', 'dport', 'flags', 'n_pkt_src', 'n_pkt_dst'

Certaine dates sont partagé pour plusieurs entrées (ex : 2021-04-09 18:42:46.997219 a 4 entrés)

Le s dans ip_s, sport est pour source et le d pour destination

Il y a des cas noté normaux dans les csv attack, besoin du csv normal ?

different types d'attaques : 

 - ['normal' 'anomaly' 'MITM' 'physical fault'] pour attack_1
 - 



In [7]:
get_columns_info('label')

Column name:  label
Number of unique values in attack:  4
Number of missing values in attack:  0
Number of values in attack:  5527407
Values in attack:  ['normal' 'anomaly' 'MITM' 'physical fault']


In [9]:
df_attack.columns

Index(['Time', 'mac_s', 'mac_d', 'ip_s', 'ip_d', 'sport', 'dport', 'proto',
       'flags', 'size', 'modbus_fn', 'n_pkt_src', 'n_pkt_dst',
       'modbus_response', 'label_n', 'label'],
      dtype='object')

In [8]:
# Remove NaN values from the column list

print("Before removing NaN values: ", df_attack1.shape)
column_NaN = ['mac_s', 'mac_d', 'ip_s', 'ip_d', 'sport', 'dport', 'flags', 'proto', 'size', 'n_pkt_src', 'n_pkt_dst','label']

df_attack1 = df_attack1.dropna(subset=column_NaN)
print("After removing NaN values: ", df_attack1.shape)

Before removing NaN values:  (5527407, 16)
After removing NaN values:  (5526894, 16)


In [9]:
# Convert the columns to the right type

column_type = ['sport', 'dport', 'flags', 'n_pkt_src', 'n_pkt_dst']

for column in column_type:
    df_attack1[column] = df_attack1[column].astype('int64')

# Remove brackets from the value of modbus_response and convert to int ignore NaN values

df_attack1['modbus_response'] = df_attack1['modbus_response'].str.replace('[','').str.replace(']','').astype('float64')

In [10]:
print(df_attack1['modbus_response'].isna().sum())

2839667


In [11]:
df_attack1['modbus_response'] = df_attack1['modbus_response'].fillna(-1) # -1 means that the value is missing
df_attack1['modbus_response'] = df_attack1['modbus_response'].astype('int64')

In [ ]:
# Remove the Time

# df_attack1 = df_attack1.drop(['Time'], axis=1)

In [12]:
# Save the cleaned dataframes

df_attack1.to_csv('/home/johan/Desktop/PDSSC/Projet/data/dataset/Network datatset/clean_csv/attack_3_cleaned.csv', index=False)

In [23]:
# KNN algorithm

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Load the cleaned dataframes

df_attack1 = pd.read_csv('/home/johan/Desktop/PDSSC/Projet/data/dataset/Network datatset/clean_csv/attack_1_cleaned.csv', delimiter=',', header=0)

In [9]:
# train_test_split

df_attack1['modbus_fn'].value_counts()

modbus_fn
Read Holding Registers Response    1430074
Read Holding Registers             1429906
Read Coils Request                 1257153
Read Coils Response                1257153
Name: count, dtype: int64

In [24]:
df_attack1.columns

Index(['mac_s', 'mac_d', 'ip_s', 'ip_d', 'sport', 'dport', 'proto', 'flags',
       'size', 'modbus_fn', 'n_pkt_src', 'n_pkt_dst', 'modbus_response',
       'label_n', 'label'],
      dtype='object')

In [25]:
def get_infos_col(col_name):
    print('Number of unique values: ', df_attack1[col_name].nunique())
    print('Number of missing values: ', df_attack1[col_name].isnull().sum())
    print('Number of values: ', df_attack1[col_name].count())
    print('Values: ', df_attack1[col_name].unique())

In [27]:
df_attack1.dtypes

mac_s              object
mac_d              object
ip_s               object
ip_d               object
sport               int64
dport               int64
proto              object
flags               int64
size                int64
modbus_fn          object
n_pkt_src           int64
n_pkt_dst           int64
modbus_response     int64
label_n             int64
label              object
dtype: object